# **Predict the news or article for reliable or not reliable using Logistic Regression**

## **Import the libraries**

In [178]:
import pandas as pd
import numpy as np
import re # regular expression
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [179]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [180]:
#stopword in english

print(stopwords.words("english"))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

## **About the dataset**

train.csv: A full training dataset with the following attributes:

- id: unique id for a news article
- title: the title of a news article
- author: author of the news article
- text: the text of the article; could be incomplete

label: a label that marks the article as potentially unreliable


> 1: unreliable

> 0: reliable


test.csv: A testing training dataset with all the same attributes at train.csv without the label.

In [181]:
news_dataset = pd.read_csv("train.csv")

In [182]:
#print the first 5 data on the dataset

news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [183]:
news_dataset['label'].value_counts()

,count
label,
1,10413
0,10387


## **Data Preprocessing**

In [184]:
news_dataset.shape

(20800, 5)

In [185]:
#check the missing value on the dataframe

news_dataset.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [186]:
#fill the missing value with the blank

news_dataset = news_dataset.fillna('')

In [187]:
#combine the news title with the news article

news_dataset['header_content'] = news_dataset['author']+' '+news_dataset['title']

In [188]:
print(news_dataset['header_content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: header_content, Length: 20800, dtype: object


In [189]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [190]:
print(X)
print(Y)

          id  ...                                     header_content
0          0  ...  Darrell Lucus House Dem Aide: We Didn’t Even S...
1          1  ...  Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2          2  ...  Consortiumnews.com Why the Truth Might Get You...
3          3  ...  Jessica Purkiss 15 Civilians Killed In Single ...
4          4  ...  Howard Portnoy Iranian woman jailed for fictio...
...      ...  ...                                                ...
20795  20795  ...  Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796  20796  ...  Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797  20797  ...  Michael J. de la Merced and Rachel Abrams Macy...
20798  20798  ...  Alex Ansary NATO, Russia To Hold Parallel Exer...
20799  20799  ...            David Swanson What Keeps the F-35 Alive

[20800 rows x 5 columns]
0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 2080

## **Stemming**

Stemming is the process of reducing a word into their root word

example:
actor, actress, acting -> **act**

In [191]:
port_stem = PorterStemmer()

In [192]:
def stemmer(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [193]:
news_dataset['header_content'] = news_dataset['header_content'].apply(stemmer)

In [194]:
print(news_dataset['header_content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: header_content, Length: 20800, dtype: object


In [195]:
#separating the new stemmer data and the label

X = news_dataset['header_content'].values
Y = news_dataset['label'].values

In [196]:
print(X)


['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [197]:
X.shape


(20800,)

In [198]:
print(Y)

[1 0 1 ... 0 1 1]


In [199]:
Y.shape

(20800,)

In [168]:
#convert the textual data to the numerical data

vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [169]:
print(X)

  (0, 267)	0.2701012497770876
  (0, 2483)	0.36765196867972083
  (0, 2959)	0.24684501285337127
  (0, 3600)	0.3598939188262558
  (0, 3792)	0.27053324808454915
  (0, 4973)	0.23331696690935097
  (0, 7005)	0.2187416908935914
  (0, 7692)	0.24785219520671598
  (0, 8630)	0.2921251408704368
  (0, 8909)	0.36359638063260746
  (0, 13473)	0.2565896679337956
  (0, 15686)	0.2848506356272864
  (1, 1497)	0.2939891562094648
  (1, 1894)	0.15521974226349364
  (1, 2223)	0.3827320386859759
  (1, 2813)	0.19094574062359204
  (1, 3568)	0.26373768806048464
  (1, 5503)	0.7143299355715573
  (1, 6816)	0.1904660198296849
  (1, 16799)	0.30071745655510157
  (2, 2943)	0.3179886800654691
  (2, 3103)	0.46097489583229645
  (2, 5389)	0.3866530551182615
  (2, 5968)	0.3474613386728292
  (2, 9620)	0.49351492943649944
  :	:
  (20797, 3643)	0.2115550061362374
  (20797, 7042)	0.21799048897828685
  (20797, 8364)	0.22322585870464115
  (20797, 8988)	0.36160868928090795
  (20797, 9518)	0.29542040034203126
  (20797, 9588)	0.17455348

## **Splitting the dataset to training and test data**

In [175]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 2)

## **Train the Model**

In [176]:
model = LogisticRegression()

In [177]:
model.fit(X_train, Y_train)

LogisticRegression()

## **Evaluation**

In [201]:
#Accuracy Score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [202]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9863581730769231


In [203]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [204]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9790865384615385


## **Making a Predictive System for the News**

In [210]:
X_new = X_test[5]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [215]:
print(Y_test[5])

1


In [212]:
#Predict few News

X_new = X_test[:10]

predictions = model.predict(X_new)

# Menampilkan hasil prediksi
for i, pred in enumerate(predictions):
    print(f"News {i+1}: {'Fake' if pred == 1 else 'Real'}")

News 1: Fake
News 2: Real
News 3: Fake
News 4: Real
News 5: Real
News 6: Fake
News 7: Real
News 8: Fake
News 9: Fake
News 10: Fake


In [213]:
print(Y_test[:10])

[1 0 1 0 0 1 0 0 1 1]


In [214]:
# Ambil 10 label asli dari Y_test
true_labels = Y_test[:10]

# Hitung jumlah prediksi yang benar
correct_predictions = sum(predictions == true_labels)

# Hitung akurasi
accuracy = correct_predictions / len(true_labels) * 100

# Tampilkan hasil
print("Predictions vs Actual Labels:")
for i, (pred, actual) in enumerate(zip(predictions, true_labels), start=1):
    print(f"News {i}: Predicted - {'Fake' if pred == 1 else 'Real'}, Actual - {'Fake' if actual == 1 else 'Real'}")

print(f"\nAccuracy: {accuracy:.2f}%")

Predictions vs Actual Labels:
News 1: Predicted - Fake, Actual - Fake
News 2: Predicted - Real, Actual - Real
News 3: Predicted - Fake, Actual - Fake
News 4: Predicted - Real, Actual - Real
News 5: Predicted - Real, Actual - Real
News 6: Predicted - Fake, Actual - Fake
News 7: Predicted - Real, Actual - Real
News 8: Predicted - Fake, Actual - Real
News 9: Predicted - Fake, Actual - Fake
News 10: Predicted - Fake, Actual - Fake

Accuracy: 90.00%
